In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution1D, concatenate, SpatialDropout1D, GlobalMaxPool1D, GlobalAvgPool1D, Embedding, \
    Conv2D, SeparableConv1D, Add, BatchNormalization, Activation, GlobalAveragePooling2D, LeakyReLU, Flatten
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, \
    Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D
from keras.layers.pooling import _GlobalPooling1D
from keras.losses import mae, sparse_categorical_crossentropy, binary_crossentropy
from keras.models import Model
from keras.datasets import mnist, cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
import keras

Using TensorFlow backend.


In [2]:
!pip install git+https://github.com/tensorflow/privacy

  Cloning https://github.com/tensorflow/privacy to c:\users\hesl\appdata\local\temp\pip-req-build-vd0j9kqm
  Created wheel for tensorflow-privacy: filename=tensorflow_privacy-0.2.2-cp37-none-any.whl size=83709 sha256=9b184d8cbe6dc9a9665cf8e2705fc0b72616cd49489f13beabf2990a6619d1c2
  Stored in directory: C:\Users\hesl\AppData\Local\Temp\pip-ephem-wheel-cache-7n5e16qa\wheels\8a\e4\14\41d16468ac11ec804bd21cfb75fc2e24f96b9e4c5af778f576
Successfully built tensorflow-privacy


  Running command git clone -q https://github.com/tensorflow/privacy 'C:\Users\hesl\AppData\Local\Temp\pip-req-build-vd0j9kqm'


In [3]:
# load data
import numpy as np
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = np.reshape(X_train, (-1, 28, 28, 1))
X_test = np.reshape(X_test, (-1, 28, 28, 1))
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
# define baseline model

11493376/11490434 [==============================] - 5s 0us/step


In [0]:
from keras.datasets import cifar10

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
shape = X_train.shape[1:]

In [4]:
def create_model2(input_shape, num_category):
  ##model building
  model = Sequential()
  #convolutional layer with rectified linear unit activation
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=input_shape))
  #32 convolution filters used each of size 3x3
  #again
  model.add(Conv2D(64, (3, 3), activation='relu'))
  #64 convolution filters used each of size 3x3
  #choose the best features via pooling
  model.add(MaxPooling2D(pool_size=(2, 2)))
  #randomly turn neurons on and off to improve convergence
  model.add(Dropout(0.25))
  #flatten since too many dimensions, we only want a classification output
  model.add(Flatten())
  #fully connected to get all relevant data
  model.add(Dense(128, activation='relu'))
  #one more dropout for convergence' sake :) 
  model.add(Dropout(0.5))
  #output a softmax to squash the matrix into output probabilities
  model.add(Dense(num_category, activation='softmax'))
  #Adaptive learning rate (adaDelta) is a popular form of gradient descent rivaled only by adam and adagrad
  #categorical ce since we have multiple classes (10) 
  model.compile(loss=keras.losses.categorical_crossentropy,
                optimizer=keras.optimizers.Adadelta(),
                metrics=['accuracy'])
  return model

from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer, DPAdamGaussianOptimizer, DPAdamOptimizer
def create_dp_model2(input_shape, num_category):
  ##model building
  dpopt = DPAdamGaussianOptimizer(l2_norm_clip=1.0, noise_multiplier=1.1, num_microbatches=128, learning_rate=0.001)
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.compat.v1.losses.Reduction.NONE)
  model = Sequential()
  #convolutional layer with rectified linear unit activation
  model.add(Conv2D(32, kernel_size=(3, 3),
                  activation='relu',
                  input_shape=input_shape))
  #32 convolution filters used each of size 3x3
  #again
  model.add(Conv2D(64, (3, 3), activation='relu'))
  #64 convolution filters used each of size 3x3
  #choose the best features via pooling
  model.add(MaxPooling2D(pool_size=(2, 2)))
  #randomly turn neurons on and off to improve convergence
  model.add(Dropout(0.25))
  #flatten since too many dimensions, we only want a classification output
  model.add(Flatten())
  #fully connected to get all relevant data
  model.add(Dense(128, activation='relu'))
  #one more dropout for convergence' sake :) 
  model.add(Dropout(0.5))
  #output a softmax to squash the matrix into output probabilities
  model.add(Dense(num_category, activation='softmax'))
  #Adaptive learning rate (adaDelta) is a popular form of gradient descent rivaled only by adam and adagrad
  #categorical ce since we have multiple classes (10) 
  model.compile(loss=loss,
                optimizer=dpopt,
                metrics=['accuracy'])
  return model


In [5]:

def create_model(shape, num_classes):
	# create model
  model = keras.Sequential([
      keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=shape),
      keras.layers.MaxPool2D(2, 1),
      keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      keras.layers.MaxPool2D(2, 1),
      keras.layers.Flatten(),
      keras.layers.Dense(32, activation='relu'),
      keras.layers.Dense(num_classes)
  ])
  model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
  return model
	# build the model

from tensorflow_privacy.privacy.optimizers.dp_optimizer import DPGradientDescentGaussianOptimizer, DPAdamGaussianOptimizer, DPAdamOptimizer


def create_dp_model(shape, num_classes):
  
  #dpopt = DPGradientDescentGaussianOptimizer(l2_norm_clip=1.0, noise_multiplier=1.1, num_microbatches=250, learning_rate=0.15)
  dpopt = DPAdamGaussianOptimizer(l2_norm_clip=1.0, noise_multiplier=1.1, num_microbatches=250, learning_rate=0.001)
  loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.compat.v1.losses.Reduction.NONE)
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(16, 8,
                            strides=2,
                            padding='same',
                            activation='relu',
                            input_shape=shape),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Conv2D(32, 4,
                            strides=2,
                            padding='valid',
                            activation='relu'),
      tf.keras.layers.MaxPool2D(2, 1),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(num_classes)
  ])
  model.compile(loss=loss,
              optimizer=dpopt,
              metrics=['accuracy'])
  return model

In [8]:
model = create_model2(X_test.shape[1:], 10)
print(model.summary())
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=250, verbose=1)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=2)
print(scores)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776  

In [6]:
model_store = []

import numpy as np
clients = 4
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
X_train, y_train = unison_shuffled_copies(X_train, y_train)

X_train_split = np.split(X_train, clients)
y_train_split = np.split(y_train, clients)

for i in range(clients):
  print('Training Model:',i)
  model = create_model2(X_train.shape[1:], 10)
  model.fit(X_train_split[i], y_train_split[i], validation_data=(X_test, y_test), epochs=10, batch_size=200, verbose=0)
  print(model.evaluate(X_test, y_test))
  model_store.append(model)
weights = [model.get_weights() for model in model_store]
epochs = 20
def avg(weights):
  new_weights = []
  for weights_list_tuple in zip(*weights):
      new_weights.append(
          [np.array(weights_).mean(axis=0)\
              for weights_ in zip(*weights_list_tuple)])
  return new_weights

for j in range(epochs):
  weights = [model.get_weights() for model in model_store]
  new_weights = avg(weights)
  global_dp_model = create_model2(X_test.shape[1:],10)
  global_dp_model.set_weights(new_weights)
  scores = global_dp_model.evaluate(X_test, y_test)
  print(j,':',scores)
  for i, model in enumerate(model_store):
    model.set_weights(new_weights)
    model.fit(X_train_split[i], y_train_split[i], validation_data=(X_test, y_test), epochs=1, batch_size=200, verbose=1)
    #print(model.evaluate(X_test, y_test))

global_dp_model = create_model2(X_test.shape[1:],10)
global_dp_model.set_weights(new_weights)
scores = global_dp_model.evaluate(X_test, y_test)

#scores = global_dp_model.evaluate(X_test, y_test)
print(scores)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Training Model: 0


10000/10000 [==============================] - 2s 213us/step
[0.08146530080835644, 0.979200005531311]
Training Model: 1
10000/10000 [==============================] - 2s 212us/step
[0.07629589683762954, 0.9807999730110168]
Training Model: 2
10000/10000 [==============================] - 2s 246us/step
[0.06830544563137977, 0.9807999730110168]
Training Model: 3
10000/10000 [==============================] - 3s 251us/step
[0.058769419142500556, 0.9840999841690063]
10000/10000 [==============================] - 2s 217us/step
0 : [2.261798006439209, 0.2973000109195709]
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [==============================] - 13s 884us/step - loss: 0.3750 - accuracy: 0.8839 - val_loss: 0.0780 - val_accuracy: 0.9772
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [==============================] - 13s 898us/step - loss: 0.3788 - accuracy: 0.8867 - val_loss: 0.0781 - val_accuracy: 0.9755
Train on 1500

15000/15000 [==============================] - 14s 960us/step - loss: 0.0413 - accuracy: 0.9863 - val_loss: 0.0390 - val_accuracy: 0.9879
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [==============================] - 14s 930us/step - loss: 0.0464 - accuracy: 0.9859 - val_loss: 0.0373 - val_accuracy: 0.9870
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [==============================] - 14s 933us/step - loss: 0.0544 - accuracy: 0.9834 - val_loss: 0.0361 - val_accuracy: 0.9882
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 2s 246us/step
9 : [0.03135017372728926, 0.9889000058174133]
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [==============================] - 14s 926us/step - loss: 0.0426 - accuracy: 0.9865 - val_loss: 0.0368 - val_accuracy: 0.9876
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [============================

Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [==============================] - 14s 953us/step - loss: 0.0269 - accuracy: 0.9912 - val_loss: 0.0411 - val_accuracy: 0.9864
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [==============================] - 14s 925us/step - loss: 0.0267 - accuracy: 0.9908 - val_loss: 0.0316 - val_accuracy: 0.9896
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
10000/10000 [==============================] - 3s 268us/step
18 : [0.028108818774940484, 0.9911999702453613]
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [==============================] - 14s 948us/step - loss: 0.0241 - accuracy: 0.9919 - val_loss: 0.0385 - val_accuracy: 0.9883
Train on 15000 samples, validate on 10000 samples
Epoch 1/1
15000/15000 [==============================] - 14s 932us/step - loss: 0.0258 - accuracy: 0.9924 - val_loss: 0.0331 - val_accuracy: 0.9891
Train on 15000 samples, validate on 100

In [8]:
!pip install -q pyyaml h5py

In [9]:
global_dp_model.save('model1.h5')

In [10]:
#global_dp_model.save('model1.h5')
converter = tf.lite.TFLiteConverter.from_keras_model_file('model1.h5')
tflite_model = converter.convert()
tflite_model_file = "model1.tflite"
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
print(interpreter.get_input_details())
def eval_model(interpreter, x_test, y_test):
  total_seen = 0
  num_correct = 0

  for img, label in zip(x_test, y_test):
    inp = img.reshape((1, 28, 28, 1))
    inp = np.float32(inp)
    #print(inp.shape)
    total_seen += 1
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_index)
    if np.argmax(predictions) == np.argmax(label):
      num_correct += 1

    if total_seen % 1000 == 0:
        print("Accuracy after %i images: %f" %
              (total_seen, float(num_correct) / float(total_seen)))
  print(num_correct, total_seen)
  return float(num_correct) / float(total_seen)

print(eval_model(interpreter, X_test, y_test))

Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.
[{'name': 'conv2d_49_input', 'index': 3, 'shape': array([ 1, 28, 28,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
Accuracy after 1000 images: 0.990000
Accuracy after 2000 images: 0.986000
Accuracy after 3000 images: 0.984000
Accuracy after 4000 images: 0.985250
Accuracy after 5000 images: 0.986400
Accuracy after 6000 images: 0.988000
Accuracy after 7000 images: 0.988857
Accuracy after 8000 images: 0.990250
Accuracy after 9000 images: 0.991111
Accuracy after 10000 images: 0.991100
9911 10000
0.9911


In [18]:
print(interpreter.size)

AttributeError: 'Interpreter' object has no attribute 'size'

In [0]:
# Please crosscheck implementation from here: https://github.com/tensorflow/privacy/tree/master/tutorials
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
def compute_epsilon():
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  sampling_probability = 1000 / 15000
  s = 10*15000 // 1000
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=1.1,
                    steps=s,
                    orders=orders)
  # Delta is set to 1e-5 because MNIST has 60000 training points.
  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]
print(compute_epsilon())

5.963887517723453


In [0]:
model = create_dp_model(X_test.shape[1:], 10)
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=250, verbose=1)
# Final evaluation of the model
weights = model.get_weights()
model = create_model(X_test.shape[1:], 10)
model.set_weights(weights)
scores = model.evaluate(X_test, y_test, verbose=1)
print(scores)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 78s 1ms/sample - loss: 3.1185 - acc: 0.3968 - val_loss: 1.6436 - val_acc: 0.6407
Epoch 2/20
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.5219 - acc: 0.6707 - val_loss: 1.2636 - val_acc: 0.7334
Epoch 3/20
60000/60000 [==============================] - 70s 1ms/sample - loss: 1.2889 - acc: 0.7334 - val_loss: 1.1422 - val_acc: 0.7509
Epoch 4/20
60000/60000 [==============================] - 72s 1ms/sample - loss: 1.2056 - acc: 0.7536 - val_loss: 1.0697 - val_acc: 0.7944
Epoch 5/20
60000/60000 [==============================] - 71s 1ms/sample - loss: 1.1946 - acc: 0.8475 - val_loss: 1.0206 - val_acc: 0.8882
Epoch 6/20
60000/60000 [==============================] - 71s 1ms/sample - loss:

KeyboardInterrupt: ignored

In [19]:
model_store = []

import numpy as np
clients = 4
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
X_train, y_train = unison_shuffled_copies(X_train, y_train)

X_train_split = np.split(X_train, clients)
y_train_split = np.split(y_train, clients)

for i in range(clients):
  print('Training Model:',i)
  model = create_dp_model(X_train.shape[1:], 10)
  model.fit(X_train_split[i], y_train_split[i], validation_data=(X_test, y_test), epochs=10, batch_size=250, verbose=1)
  #print(model.evaluate(X_test, y_test))
  model_store.append(model)
weights = [model.get_weights() for model in model_store]
epochs = 20
def avg(weights):
  new_weights = []
  for weights_list_tuple in zip(*weights):
      new_weights.append(
          np.array([np.array(weights_).mean(axis=0)\
              for weights_ in zip(*weights_list_tuple)]))
  return np.array(new_weights)

for j in range(epochs):
  weights = [model.get_weights() for model in model_store]
  new_weights = avg(weights)
  global_dp_model = create_model(X_test.shape[1:],10)
  global_dp_model.set_weights(new_weights)
  scores = global_dp_model.evaluate(X_test, y_test)
  print(j,':',scores)
  for i, model in enumerate(model_store):
    model.set_weights(new_weights)
    model.fit(X_train_split[i], y_train_split[i], validation_data=(X_test, y_test), epochs=1, batch_size=250, verbose=1)
    #print(model.evaluate(X_test, y_test))

global_dp_model = create_model(X_test.shape[1:],10)
global_dp_model.set_weights(new_weights)
scores = global_dp_model.evaluate(X_test, y_test)

#scores = global_dp_model.evaluate(X_test, y_test)
print(scores)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Training Model: 0
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 15000 samples, validate on 10000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
15000/15000 [==============================] - 212s 14ms/sample - loss: 6.0164 - acc: 0.3040 - val_loss: 2.1866 - val_acc: 0.5130
Epoch 2/10
15000/15000 [==============================] - 211s 14ms/sample - loss: 1.9294 - acc: 0.6373 - val_loss: 1.6946 - val_acc: 0.7276
Epoch 3/10
15000/15000 [==============================] - 210s 14ms/sample - loss: 1.5800 - acc: 0.7698 - val_loss: 1.3031 - val_acc: 0.8135
Epoch 4/10
15000/15000 [==============================] - 208s 14ms/sample - loss: 1.4002 - acc: 0.8175 - val_loss: 1.1895 - val_acc: 0.8442
Epoch 5/10
15000/15000 [==============================] - 211s 14ms/sample - loss: 1.2539 - acc: 0.8440 - val_loss: 1.0905 - val_acc: 0.8606
Epoch 6/10
1500

10000/10000 [==============================] - 1s 85us/step
2 : [1.1583212337493896, 0.8780999779701233]
Train on 15000 samples, validate on 10000 samples
15000/15000 [==============================] - 211s 14ms/sample - loss: 0.6813 - acc: 0.8722 - val_loss: 0.5812 - val_acc: 0.8963
Train on 15000 samples, validate on 10000 samples
15000/15000 [==============================] - 218s 15ms/sample - loss: 0.6603 - acc: 0.8783 - val_loss: 0.6246 - val_acc: 0.8886
Train on 15000 samples, validate on 10000 samples
15000/15000 [==============================] - 217s 14ms/sample - loss: 0.7215 - acc: 0.8702 - val_loss: 0.5938 - val_acc: 0.8943
Train on 15000 samples, validate on 10000 samples
10000/10000 [==============================] - 1s 83us/step
3 : [1.210689500427246, 0.9010000228881836]
Train on 15000 samples, validate on 10000 samples
15000/15000 [==============================] - 208s 14ms/sample - loss: 0.6075 - acc: 0.8939 - val_loss: 0.5298 - val_acc: 0.9100
Train on 15000 sample

Train on 15000 samples, validate on 10000 samples
10000/10000 [==============================] - 1s 90us/step
12 : [1.3247503414154054, 0.9434000253677368]
Train on 15000 samples, validate on 10000 samples
15000/15000 [==============================] - 227s 15ms/sample - loss: 0.3940 - acc: 0.9393 - val_loss: 0.3377 - val_acc: 0.9439
Train on 15000 samples, validate on 10000 samples
15000/15000 [==============================] - 225s 15ms/sample - loss: 0.3841 - acc: 0.9415 - val_loss: 0.3610 - val_acc: 0.9415
Train on 15000 samples, validate on 10000 samples
15000/15000 [==============================] - 225s 15ms/sample - loss: 0.4280 - acc: 0.9345 - val_loss: 0.3685 - val_acc: 0.9423
Train on 15000 samples, validate on 10000 samples
10000/10000 [==============================] - 1s 86us/step
13 : [1.2806301509857179, 0.9448000192642212]
Train on 15000 samples, validate on 10000 samples
15000/15000 [==============================] - 225s 15ms/sample - loss: 0.3719 - acc: 0.9426 - val

In [21]:
global_dp_model.save('model3.h5')
converter = tf.lite.TFLiteConverter.from_keras_model_file('model3.h5')
tflite_model = converter.convert()
tflite_model_file = "model3.tflite"
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
print(interpreter.get_input_details())
def eval_model(interpreter, x_test, y_test):
  total_seen = 0
  num_correct = 0

  for img, label in zip(x_test, y_test):
    inp = img.reshape((1, 28, 28, 1))
    inp = np.float32(inp)
    #print(inp.shape)
    total_seen += 1
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_index)
    if np.argmax(predictions) == np.argmax(label):
      num_correct += 1

    if total_seen % 1000 == 0:
        print("Accuracy after %i images: %f" %
              (total_seen, float(num_correct) / float(total_seen)))
  print(num_correct, total_seen)
  return float(num_correct) / float(total_seen)

print(eval_model(interpreter, X_test, y_test))

ValueError: Fetch argument <tf.Variable 'conv2d_41/kernel:0' shape=(8, 8, 1, 16) dtype=float32> cannot be interpreted as a Tensor. (Tensor Tensor("conv2d_41/kernel/Read/ReadVariableOp:0", shape=(8, 8, 1, 16), dtype=float32) is not an element of this graph.)

In [23]:
model_store = []

import numpy as np
clients = 5
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
X_train, y_train = unison_shuffled_copies(X_train, y_train)

X_train_split = np.split(X_train, clients)
y_train_split = np.split(y_train, clients)

for i in range(clients):
  print('Training Model:',i)
  model = create_dp_model(X_train.shape[1:], 10)
  model.fit(X_train_split[i], y_train_split[i], validation_data=(X_test, y_test), epochs=10, batch_size=250, verbose=1)
  #print(model.evaluate(X_test, y_test))
  model_store.append(model)
weights = [model.get_weights() for model in model_store]
epochs = 10
def avg(weights):
  new_weights = []
  for weights_list_tuple in zip(*weights):
      new_weights.append(
          np.array([np.array(weights_).mean(axis=0)\
              for weights_ in zip(*weights_list_tuple)]))
  return np.array(new_weights)

for j in range(epochs):
  weights = [model.get_weights() for model in model_store]
  new_weights = avg(weights)
  global_dp_model = create_model(X_test.shape[1:],10)
  global_dp_model.set_weights(new_weights)
  scores = global_dp_model.evaluate(X_test, y_test)
  print(j,':',scores)
  for i, model in enumerate(model_store):
    model.set_weights(new_weights)
    model.fit(X_train_split[i], y_train_split[i], validation_data=(X_test, y_test), epochs=1, batch_size=250, verbose=1)
    #print(model.evaluate(X_test, y_test))

global_dp_model = create_model(X_test.shape[1:],10)
global_dp_model.set_weights(new_weights)
scores = global_dp_model.evaluate(X_test, y_test)

#scores = global_dp_model.evaluate(X_test, y_test)
print(scores)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Training Model: 0
Train on 12000 samples, validate on 10000 samples
Epoch 1/10
12000/12000 [==============================] - 168s 14ms/sample - loss: 6.5995 - acc: 0.1801 - val_loss: 2.4262 - val_acc: 0.2827
Epoch 2/10
12000/12000 [==============================] - 174s 15ms/sample - loss: 2.2760 - acc: 0.4238 - val_loss: 1.9688 - val_acc: 0.5620
Epoch 3/10
12000/12000 [==============================] - 179s 15ms/sample - loss: 2.0481 - acc: 0.6180 - val_loss: 1.7453 - val_acc: 0.6701
Epoch 4/10
12000/12000 [==============================] - 176s 15ms/sample - loss: 1.6697 - acc: 0.6916 - val_loss: 1.5436 - val_acc: 0.7212
Epoch 5/10
12000/12000 [==============================] - 180s 15ms/sample - loss: 1.5120 - acc: 0.7509 - val_loss: 1.4384 - val_acc: 0.7769
Epoch 6/10
12000/12000 [==============================] - 179s 15ms/sample - loss: 1.4305 - acc: 0.8018 - val_loss: 1.4274 - val_acc: 0.8203
Epoch 7/10
12000/12000 [==============================] - 178s 15ms/sample - loss: 1.2

Train on 12000 samples, validate on 10000 samples
10000/10000 [==============================] - 1s 91us/step
1 : [3.03870542678833, 0.6057000160217285]
Train on 12000 samples, validate on 10000 samples
12000/12000 [==============================] - 176s 15ms/sample - loss: 0.6923 - acc: 0.7966 - val_loss: 0.4807 - val_acc: 0.8770
Train on 12000 samples, validate on 10000 samples
12000/12000 [==============================] - 170s 14ms/sample - loss: 0.7036 - acc: 0.7931 - val_loss: 0.4908 - val_acc: 0.8765
Train on 12000 samples, validate on 10000 samples
12000/12000 [==============================] - 171s 14ms/sample - loss: 0.7326 - acc: 0.7859 - val_loss: 0.5116 - val_acc: 0.8714
Train on 12000 samples, validate on 10000 samples
12000/12000 [==============================] - 173s 14ms/sample - loss: 0.7159 - acc: 0.7862 - val_loss: 0.4897 - val_acc: 0.8759
Train on 12000 samples, validate on 10000 samples
10000/10000 [==============================] - 1s 91us/step
2 : [2.9596446389

10000/10000 [==============================] - 1s 100us/step
9 : [3.4755454666137697, 0.949400007724762]
Train on 12000 samples, validate on 10000 samples
12000/12000 [==============================] - 170s 14ms/sample - loss: 0.3322 - acc: 0.9405 - val_loss: 0.3043 - val_acc: 0.9495
Train on 12000 samples, validate on 10000 samples
12000/12000 [==============================] - 172s 14ms/sample - loss: 0.3243 - acc: 0.9451 - val_loss: 0.2941 - val_acc: 0.9477
Train on 12000 samples, validate on 10000 samples
12000/12000 [==============================] - 172s 14ms/sample - loss: 0.3209 - acc: 0.9457 - val_loss: 0.3313 - val_acc: 0.9464
Train on 12000 samples, validate on 10000 samples
12000/12000 [==============================] - 169s 14ms/sample - loss: 0.3498 - acc: 0.9427 - val_loss: 0.3202 - val_acc: 0.9475
Train on 12000 samples, validate on 10000 samples
10000/10000 [==============================] - 1s 94us/step
[3.4755454666137697, 0.949400007724762]
Baseline Error: 5.06%


In [24]:
global_dp_model.save('model_n5.h5')
converter = tf.lite.TFLiteConverter.from_keras_model_file('model_n5.h5')
tflite_model = converter.convert()
tflite_model_file = "model_n5.tflite"
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
print(interpreter.get_input_details())
def eval_model(interpreter, x_test, y_test):
  total_seen = 0
  num_correct = 0

  for img, label in zip(x_test, y_test):
    inp = img.reshape((1, 28, 28, 1))
    inp = np.float32(inp)
    #print(inp.shape)
    total_seen += 1
    interpreter.set_tensor(input_index, inp)
    interpreter.invoke()
    predictions = interpreter.get_tensor(output_index)
    if np.argmax(predictions) == np.argmax(label):
      num_correct += 1

    if total_seen % 1000 == 0:
        print("Accuracy after %i images: %f" %
              (total_seen, float(num_correct) / float(total_seen)))
  print(num_correct, total_seen)
  return float(num_correct) / float(total_seen)

print(eval_model(interpreter, X_test, y_test))

INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.
[{'name': 'conv2d_21_input', 'index': 3, 'shape': array([ 1, 28, 28,  1]), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]
Accuracy after 1000 images: 0.935000
Accuracy after 2000 images: 0.930000
Accuracy after 3000 images: 0.930667
Accuracy after 4000 images: 0.930750
Accuracy after 5000 images: 0.930200
Accuracy after 6000 images: 0.937000
Accuracy after 7000 images: 0.940286
Accuracy after 8000 images: 0.944625
Accuracy after 9000 images: 0.949111
Accuracy after 10000 images: 0.949400
9494 10000
0.9494


In [ ]:
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
def compute_epsilon():
  orders = [1 + x / 10. for x in range(1, 100)] + list(range(12, 64))
  sampling_probability = 1000 / 15000
  s = 10*15000 // 1000
  rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=1.1,
                    steps=s,
                    orders=orders)
  # Delta is set to 1e-5 because MNIST has 60000 training points.
  return get_privacy_spent(orders, rdp, target_delta=1e-5)[0]
print(compute_epsilon())

In [25]:
from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent

In [27]:
compute_dp_sgd_privacy(n=25000, batch_size=32, noise_multiplier=1.3, epochs=15, delta=1e-5)

NameError: name 'compute_dp_sgd_privacy' is not defined

In [28]:
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy

In [29]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=25000, batch_size=32, noise_multiplier=1.3, epochs=15, delta=1e-5)

DP-SGD with sampling rate = 0.128% and noise_multiplier = 1.3 iterated over 11719 steps satisfies differential privacy with eps = 0.743 and delta = 1e-05.
The optimal RDP order is 21.0.


(0.7426871847684976, 21.0)